# Exploring ODF Extraction
## Sam Potter
## Current: 4/1/19

## Path and Imports

In [ ]:
import sys
import os

sys.path.extend(['C:\\Users\\potterst1\\Desktop\Repositories\BitBucket\dic',
                 'C:/Users/potterst1/Desktop/Repositories/BitBucket/dic'])
sys.path.extend(['/workspace/stpotter/git/bitbucket/dic'])
from dic import visualize
from dic import fileIO
from matplotlib import pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import patches
from matplotlib import rc
from dic import utilities
import seaborn as sns
import cv2
import matplotlib

from ipywidgets import *
from scipy import optimize as sciopt

## Magics

In [ ]:
%matplotlib inline

## Optical Mathematics

### Cylindrical scattering for single fiber

$I_{fiber}(\theta) = a_0 + a_2 \cos(2(\theta - \phi)) + a_4 \cos(4(\theta - \phi))$

### Cylindrical scattering for a distribution of fibers

$I_{dist}(\theta) = \int_{-\pi/2}^{\pi/2} I_{fiber}(\theta) PDF_{dist}(\theta) d \theta$

NOTE, CONVOLUTION OPERATION IS TREATED AS JUST SUMMATION AND NORMALIZATION IN THE WORK BELOW. CORRECT???

## Setup Single Fiber Cosine Series

### Definition of single fiber function

In [ ]:
def Ifiber(a0, a2, a4, phi, numpts=100, normalize=True):
    
    theta = np.linspace(-90, 90, numpts)
    
    vals = a0 + a2 * np.cos(np.deg2rad(2 * (theta - phi))) + a4 * np.cos(np.deg2rad(4 * (theta - phi)))
    
    if normalize:
        vals = utilities.normalize_1d(vals)
    
    return vals, theta

In [ ]:
def Ifiber_interactive(a0, a2, a4, phi, numpts=100, normalize=True):
    
    theta = np.linspace(-90, 90, numpts)
    
    vals = a0 + a2 * np.cos(np.deg2rad(2 * (theta - phi))) + a4 * np.cos(np.deg2rad(4 * (theta - phi)))
    
    if normalize:
        vals = utilities.normalize_1d(vals)
    
    fig = plt.figure(figsize=(10, 10))
    plt.plot(theta, vals, color='g', label='Intensity');
    plt.legend(prop={'size': 18});
    plt.xlabel(r'$\theta$', fontsize=18);
    plt.ylabel('Normalized Intensity (a.u.)', fontsize=18);
    plt.title('Normalized Intensity Curves of Single Fiber under Cylindrical Scattering', fontsize=18);
    plt.autoscale(enable=True, axis='x', tight=True)
    
    print('a2/a4: {}'.format(a2/a4))

### Plot a sample intensity function

In [ ]:
p = interactive(Ifiber_interactive, a0=(0, 1, 0.1), a2=(0, 0.05, 0.005), a4=(0, 0.2, 0.02), phi=(-90, 90, 5), numpts=(100, 500, 10))
display(p)

## Look at Fiber Distributions

### Setup Uniform Distribution

In [ ]:
samples = np.random.uniform(-45, 45, 10000)

In [ ]:
plt.hist(samples);

### Setup Normal Distribution

In [ ]:
samples = np.random.normal(0, 45, 10000)

In [ ]:
plt.hist(samples);

### Setup and generate samples

In [ ]:
def syntheticIdist(a0, a2, a4, splay, nsamples, distribution='uniform'):
    
    ntheta = 100
    Idist = np.zeros((nsamples, ntheta))
    for i in range(0, nsamples):
        if distribution == 'uniform':
            phi = np.random.uniform(-1*splay, splay)
        elif distribution == 'normal':
            phi = np.random.normal(0, splay)

        vals, theta_dist = Ifiber(a0, a2, a4, phi, numpts=ntheta, normalize=False)
        Idist[i, :] = vals

    Idist = np.sum(Idist, axis=0) / nsamples
    #Idist = utilities.normalize_1d(Idist)
    
    return Idist

In [ ]:
def compareI(a0, a2, a4, phi0, splay, nsamples, distribution='uniform'):
    
    ntheta = 100
    Idist = np.zeros((nsamples, ntheta))
    if distribution == 'uniform':
        phis = np.random.uniform(-1*splay, splay, nsamples)
    elif distribution == 'normal':
        phis = np.random.normal(0, splay, nsamples)
    for i in range(0, nsamples):
        phi = phis[i]
        vals, theta_dist = Ifiber(a0, a2, a4, phi, numpts=ntheta, normalize=False)
        Idist[i, :] = vals

    Idist = np.sum(Idist, axis=0) / nsamples
    #Idist = utilities.normalize_1d(Idist)
    
    Ifibers, theta_fiber = Ifiber(a0, a2, a4, phi0, numpts=ntheta, normalize=False)
    
    fig = plt.figure(figsize=(10, 10))
    plt.plot(theta_fiber, Ifibers, color='g', label='Single Fiber Intensity');
    plt.plot(theta_dist, Idist, color='r', label='Fiber Distribution Intensity');
    plt.legend(prop={'size': 14}, loc='best');
    plt.xlabel(r'$\theta$', fontsize=18);
    plt.ylabel('Intensity (a.u.)', fontsize=18);
    plt.title('Intensity Curves from Single Fiber and a Distribution of Fibers', fontsize=18);
    #plt.ylim(0, 1.25)
    plt.autoscale(enable=True, axis='x', tight=True)
    
    fig = plt.figure(figsize=(10, 10))
    plt.hist(phis)
    plt.title('Historgram of phi')
    plt.xlabel('phi')
    

In [ ]:
p = interactive(compareI, a0=(0, 1, 0.1), a2=(0, 0.05, 0.005), a4=(0, 0.2, 0.02), phi0=(-90, 90, 5), splay=(0, 90, 5), nsamples=(10, 10000, 100))
display(p)

## See if I can fit the new a0, a2, and a4 terms from the Idist

In [ ]:
def minimand(a0, a2, a4, phi, data):
    
    theta = np.linspace(-90, 90, len(data))
    
    feval = a0 + a2 * np.cos(np.deg2rad(2 * (theta - phi))) + a4 * np.cos(np.deg2rad(4 * (theta - phi)))
    
    diff = data - feval
    
    diffsq = np.square(diff, diff)
    
    ssd = np.sum(diffsq)
    
    return ssd

In [ ]:
def minfun(params, *args):

    return minimand(params[0], params[1], params[2], params[3], *args)

In [ ]:
a0 = 0
a2 = 0.0652
a4 = 0.2489
phi = 0
splay = 30
nsamples = 10000
Isingle_data, theta_single = Ifiber(a0, a2, a4, phi, normalize=False)
Idist_data = syntheticIdist(a0, a2, a4, splay, nsamples, distribution='normal')
minimand(a0, a2, a4, phi, Idist_data)

### Find the distribution terms

### Plot the data first

In [ ]:
theta = np.linspace(-90, 90, 100)
fig = plt.figure(figsize=(10, 10))
plt.plot(theta, Isingle_data, color='g', label='Reference Single Fiber Intensity')
plt.plot(theta, Idist_data, color='r', label='Fiber Distribution Intensity');
plt.legend(prop={'size': 14}, loc='best');
plt.xlabel(r'$\theta$', fontsize=18);
plt.ylabel('Intensity (a.u.)', fontsize=18);
plt.ylim(-.5, .5);

### Minimize to find the parameter values

In [ ]:
args_tup = (Idist_data)
int_x = np.array([.6, 0.025, .15, 0])
bounds = ((0, 1), (-1, 1), (0, 1), (-90, 90))
res = sciopt.minimize(minfun, int_x, args=args_tup, bounds=bounds, method='L-BFGS-B', jac='2-point')

In [ ]:
print('Final values of a0, a2, a4, phi: {}'.format(res.x))
print('Final residual: {}'.format(res.fun))

### Plot fit results

In [ ]:
a0f = res.x[0]
a2f = res.x[1]
a4f = res.x[2]
phif = res.x[3]

Ifit, theta_fit = Ifiber(a0f, a2f, a4f, phif, normalize=False)

In [ ]:
theta = np.linspace(-90, 90, 100)
fig = plt.figure(figsize=(10, 10))
plt.plot(theta, Isingle_data, color='g', label='Reference Single Fiber Intensity')
plt.plot(theta, Idist_data, color='r', label='Fiber Distribution Intensity');
plt.plot(theta, Ifit, color='k', ls='dashed', label='Fit Distribution Intensity')
plt.legend(prop={'size': 14}, loc='best');
plt.xlabel(r'$\theta$', fontsize=18);
plt.ylabel('Intensity (a.u.)', fontsize=18);
plt.ylim(-.5, .5);
print('Single Fiber a2: {}, a4: {} and a2/a4: {}'.format(a2, a4, a2/a4))
print('Fit Fiber Distribution a2: {}, a4: {} and a2/a4: {}'.format(a2f, a4f, a2f/a4f))

## Look at ODF data from SALS and compare with pSFDI a0, a2, a4

### Load in pSFDI Data

In [ ]:
pSFDI_data_path = 'C:\\Users\\potterst1\\Box Sync\\Research\\Projects\\IGA DIC pSFDI\\Experimental Data\\2D pSFDI data\\ODF Extraction\\3.24.19b\\2) pSFDI\\V_Green\data_proc.mat'
pSFDI_data = fileIO.read_mat_file(pSFDI_data_path)

phi = pSFDI_data['phi']
a0 = pSFDI_data['a0']
a2 = pSFDI_data['a2']
a4 = pSFDI_data['a4']
DOA = pSFDI_data['DOA']

# Add 180
phi = phi + 180 * np.ones(phi.shape)

### Visualize data

In [ ]:
# PD
fig = plt.figure(figsize=(10, 10))
im0 = plt.imshow(phi, cmap='hsv');
plt.title('Fiber PD');
ax = plt.gca();
divider = make_axes_locatable(ax);
cax = divider.append_axes('right', size='5%', pad=0.05)
plt.colorbar(im0, cax=cax);

In [ ]:
# a0
fig = plt.figure(figsize=(10, 10))
im0 = plt.imshow(a0, cmap='gray')
plt.title('A0');

In [ ]:
# a2
fig = plt.figure(figsize=(10, 10))
im0 = plt.imshow(a2, cmap='jet')
plt.title('a2')
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
colorlimits = (0, 0.05);
im0.set_clim(colorlimits)
plt.colorbar(im0, cax=cax);

In [ ]:
# a4
fig = plt.figure(figsize=(10, 10))
im0 = plt.imshow(a4, cmap='jet')
plt.title('a4')
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
colorlimits = (0, 0.07);
im0.set_clim(colorlimits)
plt.colorbar(im0, cax=cax);

In [ ]:
# DOA
fig = plt.figure(figsize=(10, 10))
im0 = plt.imshow(DOA, cmap='jet')
plt.title('DOA')
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
colorlimits = (0, .2);
im0.set_clim(colorlimits)
plt.colorbar(im0, cax=cax);

### Visualize a2/a4 ratio

In [ ]:
a2a4 = a2/a4
# a2/a4
fig = plt.figure(figsize=(10, 10))
im0 = plt.imshow(a2a4, cmap='jet')
plt.title('a2/a4')
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
colorlimits = (0, 2);
im0.set_clim(colorlimits)
plt.colorbar(im0, cax=cax);

### Load in SALS data

In [ ]:
SALS_data_path = 'C:\\Users\\potterst1\\Box Sync\\Research\\Projects\\IGA DIC pSFDI\\Experimental Data\\2D pSFDI data\\ODF Extraction\\SALS\\Results\\BP ODF 032919\\SALSA OUT TXT\\BP_ODF_Extraction_032919_SALSA.txt'

# Manually parse data
# Open and file and get data
f = open(SALS_data_path, 'r')
SALS_string_data = f.readlines()
SALS_string_list = [line.split('\t') for line in SALS_string_data]
f.close()

# Get variables of interest
SALS_PD = [row[2] for row in SALS_string_list]
SALS_SD = [row[6] for row in SALS_string_list]
x = [row[0] for row in SALS_string_list]
y = [row[1] for row in SALS_string_list]

# Convert strings to floats
SALS_PD = list(map(float, SALS_PD[1:]))
SALS_SD = list(map(float, SALS_SD[1:]))
x = list(map(float, x[1:]))
y = list(map(float, y[1:]))

# Convert lists to numpy arrays
SALS_PD = np.array(SALS_PD)
SALS_SD = np.array(SALS_SD)
x = np.array(x)
y = np.array(y)

In [ ]:
# Reshape the data
y_step = y[1] - y[0]
ydim = int(np.max(y) / y_step + 1)
xdim = int(len(y) / ydim)

x_2d = np.reshape(x, (xdim, ydim)).T
y_2d = np.reshape(y[::-1], (xdim, ydim)).T
PD_2d = np.reshape(SALS_PD, (xdim, ydim)).T
PD_2d = np.rad2deg(PD_2d)  # Convert radians to degrees
SD_2d = np.reshape(SALS_SD, (xdim, ydim)).T
SD_2d = np.rad2deg(SD_2d)  # Convert radians to degrees

In [ ]:
# SALS PD
fig = plt.figure(figsize=(10, 10))
im0 = plt.imshow(PD_2d, cmap='hsv')
plt.title('SALS PD')
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
colorlimits = (0, 180);
im0.set_clim(colorlimits)
plt.colorbar(im0, cax=cax);

In [ ]:
# SALS SD
fig = plt.figure(figsize=(10, 10))
im0 = plt.imshow(SD_2d, cmap='jet')
plt.title('SALS SD')
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
colorlimits = (45, 55);
im0.set_clim(colorlimits)
plt.colorbar(im0, cax=cax);

In [ ]:
# Save a file for masking a mask
name = 'C:\\Users\\potterst1\\Box Sync\\Research\\Projects\\IGA DIC pSFDI\\Experimental Data\\2D pSFDI data\\ODF Extraction\\SALS\\Results\\BP ODF 032919\\BW_Images\\color_premask.png'
plt.imsave(name, SD_2d, format='png', cmap='jet', vmin=45, vmax=55)

### Resample images

In [ ]:
# Binary mask of pSFDI image
psfdi_mask_path = 'C:\\Users\\potterst1\\Box Sync\\Research\\Projects\\IGA DIC pSFDI\\Experimental Data\\2D pSFDI data\\ODF Extraction\\3.24.19b\\2) pSFDI\\V_Green\mask.tif'
psfdi_mask = cv2.imread(psfdi_mask_path, -1)

# Binary mask of SALS image
sals_mask_path = 'C:\\Users\\potterst1\\Box Sync\\Research\\Projects\\IGA DIC pSFDI\\Experimental Data\\2D pSFDI data\\ODF Extraction\\SALS\\Results\\BP ODF 032919\\BW_Images\\Mask.tif'
sals_mask = cv2.imread(sals_mask_path, -1)

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15, 15))
im0 = ax0.imshow(psfdi_mask, cmap='gray')
ax0.set_title('Data Mask for pSFDI Imaging');

im1 = ax1.imshow(sals_mask, cmap='gray')
ax1.set_title('Data Mask for SALS Imaging');

In [ ]:
# upscale the psfdi data
# set the size of the upscaling
sals_rows, sals_columns = sals_mask.shape
psfdi_rows, psfdi_columns = psfdi_mask.shape

row_step = int(psfdi_rows / sals_rows)
col_step = int(psfdi_columns / sals_columns)

psfdi_mask_upscaled = np.zeros(sals_mask.shape)

for i in range(0, sals_rows):
    for j in range(0, sals_columns):
        subar = psfdi_mask[i * row_step: i * row_step + row_step, j * col_step: j * col_step + col_step]
        val = np.mean(subar)
        psfdi_mask_upscaled[i, j] = val

In [ ]:
# Show the mask results without thresholding
fig = plt.figure(figsize=(10, 10))
im0 = plt.imshow(psfdi_mask_upscaled, cmap='gray')
plt.title('Upscaled pSFDI Mask Before Thresholding')
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
plt.colorbar(im0, cax=cax);

In [ ]:
# Threshold mask, then plot
psfdi_mask_upscaled = psfdi_mask_upscaled >= 255
psfdi_mask_upscaled = 255 * psfdi_mask_upscaled.astype('uint8')

# Show the mask results after thresholding
fig = plt.figure(figsize=(10, 10))
im0 = plt.imshow(psfdi_mask_upscaled, cmap='gray')
plt.title('Upscaled pSFDI Mask After Thresholding')
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
plt.colorbar(im0, cax=cax);

In [ ]:
# Upscale the rest of the data (phi)
psfdi_phi_upscaled = np.zeros(sals_mask.shape)

for i in range(0, sals_rows):
    for j in range(0, sals_columns):
        subar_phi = phi[i * row_step: i * row_step + row_step, j * col_step: j * col_step + col_step]
        val_phi = np.mean(subar_phi)
        psfdi_phi_upscaled[i, j] = val_phi

In [ ]:
# Upscale the rest of the data (a0)
psfdi_a0_upscaled = np.zeros(sals_mask.shape)

for i in range(0, sals_rows):
    for j in range(0, sals_columns):
        subar_a0 = a0[i * row_step: i * row_step + row_step, j * col_step: j * col_step + col_step]
        val_a0 = np.mean(subar_a0)
        psfdi_a0_upscaled[i, j] = val_a0

In [ ]:
# Upscale the rest of the data (a2)
psfdi_a2_upscaled = np.zeros(sals_mask.shape)

for i in range(0, sals_rows):
    for j in range(0, sals_columns):
        subar_a2 = a2[i * row_step: i * row_step + row_step, j * col_step: j * col_step + col_step]
        val_a2 = np.mean(subar_a2)
        psfdi_a2_upscaled[i, j] = val_a2

In [ ]:
# Upscale the rest of the data (a4)
psfdi_a4_upscaled = np.zeros(sals_mask.shape)

for i in range(0, sals_rows):
    for j in range(0, sals_columns):
        subar_a4 = a4[i * row_step: i * row_step + row_step, j * col_step: j * col_step + col_step]
        val_a4 = np.mean(subar_a4)
        psfdi_a4_upscaled[i, j] = val_a4

In [ ]:
# Upscale the rest of the data (a2a4)
psfdi_a2a4_upscaled = np.zeros(sals_mask.shape)

for i in range(0, sals_rows):
    for j in range(0, sals_columns):
        subar_a2a4 = a2a4[i * row_step: i * row_step + row_step, j * col_step: j * col_step + col_step]
        val_a2a4 = np.mean(subar_a2a4)
        psfdi_a2a4_upscaled[i, j] = val_a2a4

### Register image data

In [ ]:
# Rotate psfdi image firts
sz = sals_mask.shape

rot_mat = cv2.getRotationMatrix2D((sz[1]/2, sz[0]/2), -90, 1)

psfdi_mask_rotated = cv2.warpAffine(psfdi_mask_upscaled, rot_mat, (sz[1], sz[0]),flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15, 15))
im0 = ax0.imshow(psfdi_mask_rotated, cmap='gray')
ax0.set_title('Data Mask for pSFDI Imaging');

im1 = ax1.imshow(sals_mask, cmap='gray')
ax1.set_title('Data Mask for SALS Imaging');

In [ ]:
# Find warp
warp_matrix = np.eye(2, 3, dtype=np.float32)
warp_mode = cv2.MOTION_EUCLIDEAN

niters = 5000
eps = 1e-5

criteria = (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS, niters, eps)

(cc, warp_matrix) = cv2.findTransformECC(sals_mask, psfdi_mask_rotated, warp_matrix, warp_mode, criteria)

In [ ]:
print('cc: {}'.format(cc))
print('warp matrix:')
print(warp_matrix)

In [ ]:
# Apply warp to psfdi_mask and plot alongside SALS mask
sz = sals_mask.shape
psfdi_mask_registered = cv2.warpAffine(psfdi_mask_rotated, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15, 15))
im0 = ax0.imshow(psfdi_mask_registered, cmap='gray')
ax0.set_title('Data Mask for pSFDI Imaging');

im1 = ax1.imshow(sals_mask, cmap='gray')
ax1.set_title('Data Mask for SALS Imaging');

In [ ]:
# Register the rest of the data (phi)
psfdi_phi_rotated = cv2.warpAffine(psfdi_phi_upscaled, rot_mat, (sz[1], sz[0]),flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
psfdi_phi_rotated = psfdi_phi_rotated + -90 * np.ones(psfdi_phi_rotated.shape)
psfdi_phi_registered = cv2.warpAffine(psfdi_phi_rotated, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

In [ ]:
# Register the rest of the data (a0)
psfdi_a0_rotated = cv2.warpAffine(psfdi_a0_upscaled, rot_mat, (sz[1], sz[0]),flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
psfdi_a0_registered = cv2.warpAffine(psfdi_a0_rotated, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

In [ ]:
# Register the rest of the data (a2)
psfdi_a2_rotated = cv2.warpAffine(psfdi_a2_upscaled, rot_mat, (sz[1], sz[0]),flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
psfdi_a2_registered = cv2.warpAffine(psfdi_a2_rotated, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

In [ ]:
# Register the rest of the data (a4)
psfdi_a4_rotated = cv2.warpAffine(psfdi_a4_upscaled, rot_mat, (sz[1], sz[0]),flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
psfdi_a4_registered = cv2.warpAffine(psfdi_a4_rotated, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

In [ ]:
# Register the rest of the data (a2a4)
psfdi_a2a4_rotated = cv2.warpAffine(psfdi_a2a4_upscaled, rot_mat, (sz[1], sz[0]),flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
psfdi_a2a4_registered = cv2.warpAffine(psfdi_a2a4_rotated, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

### Visualize phi and a2a4 vs SD data

In [ ]:
# Phi vs PD first
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15, 15))
im0 = ax0.imshow(psfdi_phi_registered, cmap='hsv')
divider = make_axes_locatable(ax0)
cax0 = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im0, cax=cax0)
ax0.set_title('pSFDI PD')

im1 = ax1.imshow(PD_2d, cmap='hsv')
divider = make_axes_locatable(ax1)
cax1 = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im1, cax=cax1)
ax1.set_title('SALS PD');

In [ ]:
# a2a4 vs SD 
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15, 15))
im0 = ax0.imshow(psfdi_a2a4_registered, cmap='jet')
divider = make_axes_locatable(ax0)
cax0 = divider.append_axes('right', size='5%', pad=0.05)
colorlimits = (0, 4);
im0.set_clim(colorlimits)
fig.colorbar(im0, cax=cax0)
ax0.set_title('pSFDI a2/a4')

im1 = ax1.imshow(SD_2d, cmap='jet')
divider = make_axes_locatable(ax1)
cax1 = divider.append_axes('right', size='5%', pad=0.05)
colorlimits = (45, 55);
im1.set_clim(colorlimits)
fig.colorbar(im1, cax=cax1)
ax1.set_title('SALS SD');

### Fit single fiber ai terms by convolving with SALS SD

In [ ]:
# Test
# Look at single fiber
a0 = psfdi_a0_registered[30, 30]
a2 = psfdi_a2_registered[30, 30]
a4 = psfdi_a4_registered[30, 30]
phi = psfdi_phi_registered[30, 30]

ivals, theta = Ifiber(a0, a2, a4, phi, normalize=False)

In [ ]:
# Test
# Plot the results
fig = plt.figure(figsize=(10, 10))
plt.plot(theta, ivals);
plt.title('pSFDI Measured Distribution I(theta)');

In [ ]:
def sals_syntheticIdist(a0, a2, a4, phi_mean, splay, nsamples, distribution='uniform'):
    
    ntheta = 100
    Idist = np.zeros((nsamples, ntheta))
    for i in range(0, nsamples):
        if distribution == 'uniform':
            phi = np.random.uniform(phi_mean - 1*splay, phi_mean + splay)
        elif distribution == 'normal':
            phi = np.random.normal(phi_mean, splay)

        vals, theta_dist = Ifiber(a0, a2, a4, phi, numpts=ntheta, normalize=False)
        Idist[i, :] = vals

    Idist = np.sum(Idist, axis=0) / nsamples
    #Idist = utilities.normalize_1d(Idist)
    
    return Idist

# minimand for finding single fiber ai from measured SALS sd and measured pSFDI fiber distribution ai
def sals_minimand(a0, a2, a4, phi_mean, sigma, psfdi_data):
    
    synthetic_Idist = sals_syntheticIdist(a0, a2, a4, phi_mean, sigma, 10000, distribution='normal')
    
    diff = psfdi_data - synthetic_Idist
    
    diffsq = np.square(diff, diff)
    
    ssd = np.sum(diffsq)
    
    return ssd

def sals_minfun(params, *args):
    
    return sals_minimand(params[0], params[1], params[2], params[3], *args)

In [ ]:
# Test
# Evaluate difference between distribution generated by convolving single fiber Itheta using ai measured from pSFDI with
# distribution of sigma from SALS measure with measured pSFDI data
measure_data = ivals
sigma = SD_2d[30, 30]

params = np.array([a0, a2, a4, phi])
args = (sigma, measure_data)

sals_minfun(params, *args)

In [ ]:
# Test
# Try to minimize to find the single fiber values that get us this distribution given sigma
args = (sigma, measure_data)
int_x = np.array([a0, a2, a4, phi])
bounds = ((0, 1), (-1, 1), (0, 1), (-90, 90))
res = sciopt.minimize(sals_minfun, int_x, args=args, bounds=None, method='L-BFGS-B', jac='2-point')

In [ ]:
print('Final values of a0, a2, a4, phi: {}'.format(res.x))
print('Initial guess: {}'.format(int_x))
print('Final residual: {}'.format(res.fun))